In [ ]:
#'cffex-host-alarm.csv' process code
#Author: 普俊韬
#Last update: 20180626

#导入pandas、numpy库
import pandas as pd
import numpy as np


#TODO：进行原始数据的分割处理

#读取原始数据（GBK编码，数据不带列标题）
data = pd.read_csv('raw_data\cffex-host-alarm.csv',encoding = 'GBK', header = None)

#分割数据字段，分隔符'|||'（'[|]+'),分割后扩展列
data_processed = data[0].str.split('[|]+',expand = True)

#插入列标题
data_processed.columns = ['node_name', 'node_alias', 'component', 'category', 'alarm_count', 'first_time', 'last_time', 'alarm_level', 'alarm_content']


#TODO：进行'component'字段的处理

#将'component'字段提取出来作为一个DataFrame
data_component = data_processed.loc[:,['component']]

#去掉重复数据
data_component_processed = data_component.drop_duplicates()

#插入id列，编号从1开始
data_component_processed['id'] = range(1,len(data_component_processed) + 1)

#将列顺序调整为['id', 'component']
data_component_processed = data_component_processed[['id','component']]

#将处理后结果写入'cffex-host-alarm-component.csv'（不带行标签，GBK编码）
data_component_processed.to_csv('output_data\cffex-host-alarm-component.csv', index = 0, encoding = 'GBK')


#TODO：进行'category'字段的处理

#将'category'字段提取出来作为一个DataFrame
data_category = data_processed.loc[:,['category']]

#去掉重复数据
data_category_processed = data_category.drop_duplicates()

#插入id列，编号从1开始
data_category_processed['id'] = range(1,len(data_category_processed) + 1)

#将列顺序调整为['id', 'category']
data_category_processed = data_category_processed[['id','category']]

#将处理后结果写入'cffex-host-alarm-category.csv'（不带行标签，GBK编码）
data_category_processed.to_csv('output_data\cffex-host-alarm-category.csv', index = 0, encoding = 'GBK')


#TODO：将'component'和'category'字段替换为对应的'id'值，方便后续的数据处理

#对'component'字段进行查找和替换
data_processed['component'] = data_processed['component'].replace(data_component_processed['component'].tolist(),data_component_processed['id'].tolist())

#对'category'字段进行查找和替换
data_processed['category'] = data_processed['category'].replace(data_category_processed['category'].tolist(),data_category_processed['id'].tolist())

#将处理后结果写入'cffex-host-alarm-processed.csv'（不带行标签，GBK编码）
data_processed.to_csv('output_data\cffex-host-alarm-processed.csv', index = 0, encoding = 'GBK')



In [5]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


rcParams['figure.figsize'] = 15,6

dateparse = lambda dates: pd.datetime.strptime(dates,'%Y%m%d%H')

data = pd.read_csv(r'E:\HomeMadeSoftware\Cffex_Data_Analysis\output_data\cffex-host-info\2018_alarmsvr1_hourly_cpu.csv',
                   encoding='UTF-8',parse_dates=['archour'],index_col = 'archour',date_parser=dateparse)

ts_maxvalue = data['maxvalue']
ts_maxvalue_log = np.log(ts_maxvalue)


# 稳定性检测
def test_stationarity(timeseries):
    rolmean = timeseries.rolling(window = 168).mean()
    rolstd = timeseries.rolling(window = 168).std()
    
    fig = plt.figure()
    fig.add_subplot()
    orig = plt.plot(timeseries, color = 'blue', label = 'Original')
    mean = plt.plot(rolmean, color = 'red', label = 'rolling mean')
    std = plt.plot(rolstd,color = 'black', label = 'Rolling standard deviation')
    
    plt.legend(loc = 'best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show(block=False)
    
    
    #dickey-Fuller test:
    
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries,autolag='AIC')
    
    dfoutput = pd.Series(dftest[0:4],index = ['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical value (%s)'%key] = value
        
    print (dfoutput)
        
        
        
# 时间序列分解
def decompose(timeseries):
    decomposition = seasonal_decompose(timeseries,freq=168)
    
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid
    
    plt.subplot(411)
    plt.plot(ts_maxvalue_log,label='Original')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend,label='Trend')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual,label='Residuals')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()
    
    return trend,seasonal,residual

# 时间序列分解调用
# trend,seasonal,residual = decompose(ts_maxvalue_log)
# residual.dropna(inplace=True)
# test_stationarity(residual)


# 自相关、偏自相关图
def ACF_PACF(timeseries):
    lag_acf = acf(timeseries, nlags = 20)
    lag_pacf = pacf(timeseries, nlags = 20, method='ols')

    plt.subplot(121)
    plt.plot(lag_acf)
    plt.axhline(y=0,linestyle='--',color='gray')
    plt.axhline(y=-1.96/np.sqrt(len(timeseries)),linestyle='--',color='gray')
    plt.axhline(y=1.96/np.sqrt(len(timeseries)),linestyle='--',color='gray')
    plt.title('Autocorrelation Function')

    plt.subplot(122)
    plt.plot(lag_pacf)
    plt.axhline(y=0,linestyle='--',color='gray')
    plt.axhline(y=-1.96/np.sqrt(len(timeseries)),linestyle='--',color='gray')
    plt.axhline(y=1.96/np.sqrt(len(timeseries)),linestyle='--',color='gray')
    plt.title('Partial Autocorrelation Function')
    plt.tight_layout()
    
    plt.show()


# ARIMA模型
def ARIMA_Pre(timeseries,order):
    model = ARIMA(timeseries,order)
    results_ARIMA= model.fit(disp=-1)
    plt.plot(timeseries, color = 'blue')
    plt.plot(results_ARIMA.fittedvalues,color='red')
    plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-timeseries)**2))
    plt.show(block=False)
    
    return results_ARIMA.fittedvalues

# MA模型
def MA(timeseries, ordered_window):
    moving_avg = timeseries.rolling(window = ordered_window).mean()
    plt.plot(timeseries, color = 'blue')
    plt.plot(moving_avg, color = 'red')
    plt.title('Moving Average')
    plt.show(block=False)
    return moving_avg

# ES模型
def ES(alpha, timeseries):
    ES_TS = timeseries.copy(deep=True)
    ES_TS[ES_TS!=0] = 0.0
    for i in range(1, len(ES_TS)):
        ES_TS[i] = alpha*timeseries[i]+(1-alpha)*ES_TS[i-1]
    
    plt.plot(ts_maxvalue_log, color = 'blue')
    plt.plot(ES_TS, color = 'red')
    plt.title('Exponential Smoothing')
    plt.show(block=False)
    return ES_TS

    
# 回归预测评价
def evaluate(timeseries,results):
    print('MSE: %.4f'% mean_squared_error(timeseries,results))
    print('RMSE: %.4f'% (mean_squared_error(timeseries,results))**0.5)
    print('MAE: %.4f'% mean_absolute_error(timeseries,results))
    print('R-Square: %.4f'% r2_score(timeseries,results))
    

# 稳定性检测
test_stationarity(ts_maxvalue_log)

# ACF-PACF
print('ACF-PACF:')
ACF_PACF(ts_maxvalue_log)

# 时间序列分解
print('Time Series Decompose:')
decompose(ts_maxvalue_log)

# MA结果
print('MA(Moving Average):')
MA_results = MA(ts_maxvalue_log, 24)
nan = np.isnan(MA_results)
MA_results[nan] = 0
evaluate(ts_maxvalue_log, MA_results)

# ARIMA结果
print('ARIMA:')
ARIMA_results = ARIMA_Pre(ts_maxvalue_log,(1,0,1))
evaluate(ts_maxvalue_log, ARIMA_results)

# ES结果
print('ES[1]:')
alpha = .70#设置alphe，即平滑系数
ES_results = ES(alpha,ts_maxvalue_log)
evaluate(ts_maxvalue_log, ES_results)
print(ts_maxvalue_log)
print(ES_results)
print('ES[2]:')
ES2_results = ES(alpha, ES_results)
evaluate(ts_maxvalue_log, ES2_results)

ACF-PACF:
Time Series Decompose:


ValueError: operands could not be broadcast together with shapes (5,) (333,) 

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import os, sys, json, csv, re

rcParams['figure.figsize'] = 15,6

# ARIMA模型
def ARIMA(timeseries,order):
    model = ARIMA(timeseries,order)
    results_ARIMA= model.fit(disp=-1)
    return results_ARIMA.fittedvalues

# MA模型
def MA(timeseries, ordered_window):
    moving_avg = timeseries.rolling(window = ordered_window).mean()
    return moving_avg

# ES模型
def ES(alpha, timeseries):
    ES_TS = timeseries.copy(deep=True)
    ES_TS[ES_TS!=0] = 0.0
    for i in range(1, len(ES_TS)):
        ES_TS[i] = alpha*timeseries[i]+(1-alpha)*ES_TS[i-1]
    return ES_TS

    
# 回归预测评价
def evaluate(timeseries,results):
    print('MSE: %.4f'% mean_squared_error(timeseries,results))
    print('RMSE: %.4f'% (mean_squared_error(timeseries,results))**0.5)
    print('MAE: %.4f'% mean_absolute_error(timeseries,results))
    print('R-Square: %.4f'% r2_score(timeseries,results))
    return mean_squared_error(timeseries,results),mean_squared_error(timeseries,results)**0.5,mean_absolute_error(timeseries,results),r2_score(timeseries,results)


csv_dir = r'E:\HomeMadeSoftware\Cffex_Data_Analysis\output_data\cffex-host-info'
dateparse = lambda dates: pd.datetime.strptime(dates,'%Y%m%d%H')
f_list = os.listdir(csv_dir)
predict_result = pd.DataFrame(columns=['file','MA_maxvalue_MSE','MA_maxvalue_RMSE','MA_maxvalue_MAE','MA_maxvalue_R-Square',
                                          'MA_minvalue_MSE','MA_minvalue_RMSE','MA_minvalue_MAE','MA_minvalue_R-Square',
                                          'ES1_maxvalue_MSE','ES1_maxvalue_RMSE','ES1_maxvalue_MAE','ES1_maxvalue_R-Square',
                                          'ES1_minvalue_MSE','ES1_minvalue_RMSE','ES1_minvalue_MAE','ES1_minvalue_R-Square',
                                          'ES2_maxvalue_MSE','ES2_maxvalue_RMSE','ES2_maxvalue_MAE','ES2_maxvalue_R-Square',
                                          'ES2_minvalue_MSE','ES2_minvalue_RMSE','ES2_minvalue_MAE','ES2_minvalue_R-Square',])

for i in f_list:
    data = pd.read_csv(os.path.join(csv_dir,i),encoding='UTF-8',parse_dates=['archour'],index_col = 'archour',date_parser=dateparse)
    ts_maxvalue = data['maxvalue']
    ts_minvalue = data['minvalue']
    
    print(i)
    #MA-Max
    MA_Max_results = MA(ts_maxvalue, 24)
    nan = np.isnan(MA_Max_results)
    MA_Max_results[nan] = 0
    inf = np.isinf(MA_Max_results)
    MA_Max_results[inf] = 0
    max_MA_indicator_MSE,max_MA_indicator_RMSE,max_MA_indicator_MAE,max_MA_indicator_RSquare = evaluate(ts_maxvalue, MA_Max_results)
    #MA-Min
    MA_Min_results = MA(ts_minvalue, 24)
    nan = np.isnan(MA_Min_results)
    MA_Min_results[nan] = 0
    inf = np.isinf(MA_Min_results)
    MA_Min_results[inf] = 0
    min_MA_indicator_MSE,min_MA_indicator_RMSE,min_MA_indicator_MAE,min_MA_indicator_RSquare = evaluate(ts_minvalue, MA_Min_results)
    
#     #ARIMA-Max
#     ARIMA_Max_results = ARIMA(ts_maxvalue_log,(2,0,1))
#     max_ARIMA_indicator_MSE,max_ARIMA_indicator_RMSE,max_ARIMA_indicator_MAE,max_ARIMA_indicator_RSquare = evaluate(ts_maxvalue_log, ARIMA_Max_results)
#     #ARIMA-Min
#     ARIMA_Min_results = ARIMA(ts_minvalue_log,(2,0,1))
#     min_ARIMA_indicator_MSE,min_ARIMA_indicator_RMSE,min_ARIMA_indicator_MAE,min_ARIMA_indicator_RSquare = evaluate(ts_maxvalue_log, ARIMA_Max_results)
    
    #ES1-Max
    alpha = .70#设置alphe，即平滑系数
    ES1_Max_results = ES(alpha,ts_maxvalue)
    max_ES1_indicator_MSE,max_ES1_indicator_RMSE,max_ES1_indicator_MAE,max_ES1_indicator_RSquare = evaluate(ts_maxvalue, ES1_Max_results)
    #ES1-Min
    alpha = .70#设置alphe，即平滑系数
    ES1_Min_results = ES(alpha,ts_minvalue)
    min_ES1_indicator_MSE,min_ES1_indicator_RMSE,min_ES1_indicator_MAE,min_ES1_indicator_RSquare = evaluate(ts_minvalue, ES1_Min_results)

    #ES2-Max
    alpha = .70#设置alphe，即平滑系数
    ES2_Max_results = ES(alpha,ES1_Max_results)
    max_ES2_indicator_MSE,max_ES2_indicator_RMSE,max_ES2_indicator_MAE,max_ES2_indicator_RSquare = evaluate(ts_maxvalue, ES2_Max_results)
    #ES2-Min
    alpha = .70#设置alphe，即平滑系数
    ES2_Min_results = ES(alpha,ES1_Min_results)
    min_ES2_indicator_MSE,min_ES2_indicator_RMSE,min_ES2_indicator_MAE,min_ES2_indicator_RSquare = evaluate(ts_minvalue, ES2_Min_results)
    
#     写入csv
    predict_result = predict_result.append({'file':os.path.splitext(i)[0],
                                                  'MA_maxvalue_MSE':max_MA_indicator_MSE,
                                                  'MA_maxvalue_RMSE':max_MA_indicator_RMSE,
                                                  'MA_maxvalue_MAE':max_MA_indicator_MAE,
                                                  'MA_maxvalue_R-Square':max_MA_indicator_RSquare,
                                                  'MA_minvalue_MSE':min_MA_indicator_MSE,
                                                  'MA_minvalue_RMSE':min_MA_indicator_RMSE,
                                                  'MA_minvalue_MAE':min_MA_indicator_MAE,
                                                  'MA_minvalue_R-Square':min_MA_indicator_RSquare,
                                                  'ES1_maxvalue_MSE':max_ES1_indicator_MSE,
                                                  'ES1_maxvalue_RMSE':max_ES1_indicator_RMSE,
                                                  'ES1_maxvalue_MAE':max_ES1_indicator_MAE,
                                                  'ES1_maxvalue_R-Square':max_ES1_indicator_RSquare,
                                                  'ES1_minvalue_MSE':min_ES1_indicator_MSE,
                                                  'ES1_minvalue_RMSE':min_ES1_indicator_RMSE,
                                                  'ES1_minvalue_MAE':min_ES1_indicator_MAE,
                                                  'ES1_minvalue_R-Square':min_ES1_indicator_RSquare,
                                                  'ES2_maxvalue_MSE':max_ES2_indicator_MSE,
                                                  'ES2_maxvalue_RMSE':max_ES2_indicator_RMSE,
                                                  'ES2_maxvalue_MAE':max_ES2_indicator_MAE,
                                                  'ES2_maxvalue_R-Square':max_ES2_indicator_RSquare,
                                                  'ES2_minvalue_MSE':min_ES2_indicator_MSE,
                                                  'ES2_minvalue_RMSE':min_ES2_indicator_RMSE,
                                                  'ES2_minvalue_MAE':min_ES2_indicator_MAE,
                                                  'ES2_minvalue_R-Square':min_ES2_indicator_RSquare},ignore_index = True)
    
predict_result.to_csv('output_data\TS_predict_result.csv', index = 0, encoding = 'UTF-8')
    
    

2018_alarmsvr1_hourly_cpu.csv
MSE: 1.5066
RMSE: 1.2274
MAE: 0.8280
R-Square: 0.1317
MSE: 0.3574
RMSE: 0.5978
MAE: 0.4785
R-Square: 0.1911
MSE: 0.1485
RMSE: 0.3854
MAE: 0.1865
R-Square: 0.9144
MSE: 0.0114
RMSE: 0.1066
MAE: 0.0503
R-Square: 0.9743
MSE: 0.3992
RMSE: 0.6318
MAE: 0.3079
R-Square: 0.7699
MSE: 0.0366
RMSE: 0.1912
MAE: 0.0942
R-Square: 0.9172
2018_alarmsvr1_hourly_disk.csv
MSE: 108.8923
RMSE: 10.4351
MAE: 9.0139
R-Square: 0.0618
MSE: 108.9049
RMSE: 10.4358
MAE: 9.0130
R-Square: 0.0618
MSE: 20.4830
RMSE: 4.5258
MAE: 3.8528
R-Square: 0.8235
MSE: 20.4965
RMSE: 4.5273
MAE: 3.8530
R-Square: 0.8234
MSE: 51.9818
RMSE: 7.2098
MAE: 6.1940
R-Square: 0.5521
MSE: 52.0058
RMSE: 7.2115
MAE: 6.1939
R-Square: 0.5520
2018_alarmsvr1_hourly_mem.csv
MSE: 6.4482
RMSE: 2.5393
MAE: 1.7954
R-Square: 0.1292
MSE: 4.7511
RMSE: 2.1797
MAE: 1.4600
R-Square: 0.1021
MSE: 0.2244
RMSE: 0.4737
MAE: 0.1863
R-Square: 0.9697
MSE: 0.2055
RMSE: 0.4534
MAE: 0.1783
R-Square: 0.9612
MSE: 0.7234
RMSE: 0.8505
MAE: 0.368

MSE: 0.0173
RMSE: 0.1315
MAE: 0.0084
R-Square: 0.5646
MSE: 0.0217
RMSE: 0.1472
MAE: 0.0209
R-Square: 0.7296
MSE: 0.0214
RMSE: 0.1462
MAE: 0.0132
R-Square: 0.4617
2018_bizeib2_hourly_cpu.csv
MSE: 5.2373
RMSE: 2.2885
MAE: 0.8688
R-Square: 0.0849
MSE: 0.0342
RMSE: 0.1848
MAE: 0.1111
R-Square: 0.7117
MSE: 0.6941
RMSE: 0.8331
MAE: 0.2676
R-Square: 0.8787
MSE: 0.0030
RMSE: 0.0545
MAE: 0.0340
R-Square: 0.9750
MSE: 1.8802
RMSE: 1.3712
MAE: 0.4559
R-Square: 0.6715
MSE: 0.0072
RMSE: 0.0850
MAE: 0.0568
R-Square: 0.9390
2018_bizeib2_hourly_disk.csv
MSE: 293.3339
RMSE: 17.1270
MAE: 13.3842
R-Square: 0.0128
MSE: 292.8689
RMSE: 17.1134
MAE: 13.3760
R-Square: 0.0128
MSE: 54.4175
RMSE: 7.3768
MAE: 5.8429
R-Square: 0.8169
MSE: 54.3349
RMSE: 7.3712
MAE: 5.8393
R-Square: 0.8168
MSE: 138.8055
RMSE: 11.7816
MAE: 9.3857
R-Square: 0.5328
MSE: 138.5916
RMSE: 11.7725
MAE: 9.3799
R-Square: 0.5328
2018_bizeib2_hourly_mem.csv
MSE: 0.3572
RMSE: 0.5977
MAE: 0.0874
R-Square: -5.1063
MSE: 0.3549
RMSE: 0.5957
MAE: 0.07

MSE: 35.4145
RMSE: 5.9510
MAE: 4.3297
R-Square: 0.8551
MSE: 97.2943
RMSE: 9.8638
MAE: 7.3264
R-Square: 0.6026
MSE: 97.1400
RMSE: 9.8560
MAE: 7.3210
R-Square: 0.6027
2018_bizses3_hourly_mem.csv
MSE: 0.1229
RMSE: 0.3506
MAE: 0.0556
R-Square: -2.6441
MSE: 0.1216
RMSE: 0.3487
MAE: 0.0372
R-Square: -6.1532
MSE: 0.0062
RMSE: 0.0790
MAE: 0.0094
R-Square: 0.8151
MSE: 0.0059
RMSE: 0.0765
MAE: 0.0036
R-Square: 0.6557
MSE: 0.0077
RMSE: 0.0878
MAE: 0.0154
R-Square: 0.7713
MSE: 0.0071
RMSE: 0.0843
MAE: 0.0055
R-Square: 0.5815
2018_bizsvr1_hourly_cpu.csv
MSE: 3.4580
RMSE: 1.8596
MAE: 0.8288
R-Square: 0.2627
MSE: 0.0337
RMSE: 0.1837
MAE: 0.1128
R-Square: 0.8700
MSE: 0.5131
RMSE: 0.7163
MAE: 0.2927
R-Square: 0.8906
MSE: 0.0022
RMSE: 0.0471
MAE: 0.0252
R-Square: 0.9914
MSE: 1.3605
RMSE: 1.1664
MAE: 0.4868
R-Square: 0.7099
MSE: 0.0057
RMSE: 0.0755
MAE: 0.0426
R-Square: 0.9781
2018_bizsvr1_hourly_disk.csv
MSE: 60.3840
RMSE: 7.7707
MAE: 6.4429
R-Square: 0.2116
MSE: 60.3163
RMSE: 7.7664
MAE: 6.4385
R-Squar

MSE: 0.0301
RMSE: 0.1735
MAE: 0.0676
R-Square: 0.9834
MSE: 1.0635
RMSE: 1.0313
MAE: 0.4309
R-Square: 0.8010
MSE: 0.0818
RMSE: 0.2860
MAE: 0.1162
R-Square: 0.9549
2018_config3_hourly_disk.csv
MSE: 442.0983
RMSE: 21.0261
MAE: 17.8441
R-Square: 0.0480
MSE: 441.2078
RMSE: 21.0049
MAE: 17.8284
R-Square: 0.0479
MSE: 65.5601
RMSE: 8.0969
MAE: 6.5351
R-Square: 0.8588
MSE: 65.4761
RMSE: 8.0917
MAE: 6.5303
R-Square: 0.8587
MSE: 181.1186
RMSE: 13.4580
MAE: 11.2016
R-Square: 0.6100
MSE: 180.8521
RMSE: 13.4481
MAE: 11.1931
R-Square: 0.6097
2018_config3_hourly_mem.csv
MSE: 0.6128
RMSE: 0.7828
MAE: 0.6518
R-Square: 0.0030
MSE: 0.4432
RMSE: 0.6658
MAE: 0.5127
R-Square: -0.0120
MSE: 0.0231
RMSE: 0.1518
MAE: 0.0791
R-Square: 0.9625
MSE: 0.0202
RMSE: 0.1421
MAE: 0.0676
R-Square: 0.9539
MSE: 0.0716
RMSE: 0.2675
MAE: 0.1517
R-Square: 0.8836
MSE: 0.0612
RMSE: 0.2475
MAE: 0.1296
R-Square: 0.8602
2018_config4_hourly_cpu.csv
MSE: 17.7528
RMSE: 4.2134
MAE: 1.8044
R-Square: 0.1977
MSE: 0.4195
RMSE: 0.6477
MAE: 0

MSE: 0.0190
RMSE: 0.1378
MAE: 0.0132
R-Square: 0.9210
2018_excheib2_hourly_cpu.csv
MSE: 5.2953
RMSE: 2.3011
MAE: 0.8805
R-Square: 0.0814
MSE: 0.0307
RMSE: 0.1753
MAE: 0.1042
R-Square: 0.7235
MSE: 0.6983
RMSE: 0.8357
MAE: 0.2707
R-Square: 0.8789
MSE: 0.0029
RMSE: 0.0535
MAE: 0.0325
R-Square: 0.9742
MSE: 1.8935
RMSE: 1.3760
MAE: 0.4615
R-Square: 0.6715
MSE: 0.0068
RMSE: 0.0822
MAE: 0.0541
R-Square: 0.9392
2018_excheib2_hourly_disk.csv
MSE: 230.9636
RMSE: 15.1975
MAE: 12.1607
R-Square: 0.0131
MSE: 230.5537
RMSE: 15.1840
MAE: 12.1524
R-Square: 0.0131
MSE: 43.2619
RMSE: 6.5774
MAE: 5.3209
R-Square: 0.8151
MSE: 43.1882
RMSE: 6.5718
MAE: 5.3173
R-Square: 0.8151
MSE: 109.9970
RMSE: 10.4879
MAE: 8.5414
R-Square: 0.5300
MSE: 109.8073
RMSE: 10.4789
MAE: 8.5355
R-Square: 0.5300
2018_excheib2_hourly_mem.csv
MSE: 0.3270
RMSE: 0.5718
MAE: 0.0811
R-Square: -2.7209
MSE: 0.3195
RMSE: 0.5653
MAE: 0.0714
R-Square: -6.1673
MSE: 0.0162
RMSE: 0.1274
MAE: 0.0113
R-Square: 0.8153
MSE: 0.0153
RMSE: 0.1238
MAE: 

...

MSE: 0.0046
RMSE: 0.0678
MAE: 0.0359
R-Square: 0.9535
2018_w1_multimon2_hourly_disk.csv
MSE: 54.5394
RMSE: 7.3851
MAE: 5.4799
R-Square: 0.0223
MSE: 54.4117
RMSE: 7.3764
MAE: 5.4729
R-Square: 0.0223
MSE: 9.8454
RMSE: 3.1377
MAE: 2.5762
R-Square: 0.8235
MSE: 9.8200
RMSE: 3.1337
MAE: 2.5726
R-Square: 0.8235
MSE: 25.4922
RMSE: 5.0490
MAE: 4.0379
R-Square: 0.5430
MSE: 25.4287
RMSE: 5.0427
MAE: 4.0324
R-Square: 0.5431
2018_w1_multimon2_hourly_mem.csv
MSE: 0.3018
RMSE: 0.5493
MAE: 0.0702
R-Square: -3.3974
MSE: 0.3007
RMSE: 0.5483
MAE: 0.0715
R-Square: -9.0504
MSE: 0.0144
RMSE: 0.1199
MAE: 0.0089
R-Square: 0.7904
MSE: 0.0140
RMSE: 0.1185
MAE: 0.0072
R-Square: 0.5305
MSE: 0.0174
RMSE: 0.1321
MAE: 0.0138
R-Square: 0.7458
MSE: 0.0172
RMSE: 0.1313
MAE: 0.0111
R-Square: 0.4236
2018_w1_multimon_hourly_cpu.csv
MSE: 0.1862
RMSE: 0.4316
MAE: 0.2384
R-Square: 0.5624
MSE: 0.0293
RMSE: 0.1711
MAE: 0.0951
R-Square: 0.8031
MSE: 0.0231
RMSE: 0.1521
MAE: 0.0792
R-Square: 0.9457
MSE: 0.0021
RMSE: 0.0461
MAE: 0

MSE: 0.0041
RMSE: 0.0642
MAE: 0.0130
R-Square: -0.4851
2018_webdb1_hourly_cpu.csv
MSE: 0.1256
RMSE: 0.3544
MAE: 0.2065
R-Square: 0.0654
MSE: 0.0074
RMSE: 0.0860
MAE: 0.0367
R-Square: 0.2062
MSE: 0.0188
RMSE: 0.1372
MAE: 0.0758
R-Square: 0.8599
MSE: 0.0004
RMSE: 0.0208
MAE: 0.0082
R-Square: 0.9534
MSE: 0.0492
RMSE: 0.2219
MAE: 0.1242
R-Square: 0.6338
MSE: 0.0010
RMSE: 0.0312
MAE: 0.0138
R-Square: 0.8953
2018_webdb1_hourly_disk.csv
MSE: 470.3729
RMSE: 21.6881
MAE: 18.4217
R-Square: 0.0732
MSE: 469.3588
RMSE: 21.6647
MAE: 18.4010
R-Square: 0.0732
MSE: 63.9154
RMSE: 7.9947
MAE: 5.7582
R-Square: 0.8741
MSE: 63.8213
RMSE: 7.9888
MAE: 5.7522
R-Square: 0.8740
MSE: 169.6314
RMSE: 13.0243
MAE: 9.5778
R-Square: 0.6658
MSE: 169.3661
RMSE: 13.0141
MAE: 9.5676
R-Square: 0.6656
2018_webdb1_hourly_mem.csv
MSE: 1.5774
RMSE: 1.2560
MAE: 0.1186
R-Square: -154.0353
MSE: 1.5764
RMSE: 1.2555
MAE: 0.1155
R-Square: -180.4890
MSE: 0.0753
RMSE: 0.2745
MAE: 0.0087
R-Square: -6.4045
MSE: 0.0753
RMSE: 0.2744
MAE: 

MSE: 13.3568
RMSE: 3.6547
MAE: 3.2489
R-Square: 0.5235
MSE: 13.3145
RMSE: 3.6489
MAE: 3.2435
R-Square: 0.5235
2018_web_server2_hourly_mem.csv
MSE: 0.0624
RMSE: 0.2497
MAE: 0.0398
R-Square: 0.2820
MSE: 0.0616
RMSE: 0.2481
MAE: 0.0323
R-Square: 0.1783
MSE: 0.0030
RMSE: 0.0547
MAE: 0.0051
R-Square: 0.9656
MSE: 0.0028
RMSE: 0.0527
MAE: 0.0030
R-Square: 0.9629
MSE: 0.0037
RMSE: 0.0611
MAE: 0.0083
R-Square: 0.9570
MSE: 0.0034
RMSE: 0.0587
MAE: 0.0049
R-Square: 0.9541
2018_web_tranprod1_hourly_cpu.csv
MSE: 3.9223
RMSE: 1.9805
MAE: 0.7285
R-Square: 0.0658
MSE: 0.0142
RMSE: 0.1190
MAE: 0.0705
R-Square: 0.8113
MSE: 0.5840
RMSE: 0.7642
MAE: 0.2569
R-Square: 0.8609
MSE: 0.0011
RMSE: 0.0328
MAE: 0.0191
R-Square: 0.9857
MSE: 1.5396
RMSE: 1.2408
MAE: 0.4283
R-Square: 0.6333
MSE: 0.0026
RMSE: 0.0507
MAE: 0.0320
R-Square: 0.9658
2018_web_tranprod1_hourly_disk.csv
MSE: 65.7372
RMSE: 8.1078
MAE: 5.9465
R-Square: 0.0131
MSE: 65.5684
RMSE: 8.0974
MAE: 5.9383
R-Square: 0.0131
MSE: 12.2376
RMSE: 3.4982
MAE: 

MSE: 0.0008
RMSE: 0.0286
MAE: 0.0153
R-Square: 0.9587
2018_yww_monitor2_hourly_disk.csv
MSE: 14.7220
RMSE: 3.8369
MAE: 3.2933
R-Square: 0.0708
MSE: 14.7156
RMSE: 3.8361
MAE: 3.2918
R-Square: 0.0708
MSE: 2.5464
RMSE: 1.5958
MAE: 1.3078
R-Square: 0.8393
MSE: 2.5449
RMSE: 1.5953
MAE: 1.3071
R-Square: 0.8393
MSE: 6.8096
RMSE: 2.6095
MAE: 2.2211
R-Square: 0.5702
MSE: 6.8057
RMSE: 2.6088
MAE: 2.2199
R-Square: 0.5703
2018_yww_monitor2_hourly_mem.csv
MSE: 0.0714
RMSE: 0.2672
MAE: 0.0363
R-Square: 0.9355
MSE: 0.0700
RMSE: 0.2646
MAE: 0.0366
R-Square: 0.9361
MSE: 0.0033
RMSE: 0.0577
MAE: 0.0025
R-Square: 0.9970
MSE: 0.0033
RMSE: 0.0577
MAE: 0.0026
R-Square: 0.9970
MSE: 0.0040
RMSE: 0.0635
MAE: 0.0039
R-Square: 0.9964
MSE: 0.0040
RMSE: 0.0635
MAE: 0.0039
R-Square: 0.9963
2018_zzdi1_hourly_cpu.csv
MSE: 81.4401
RMSE: 9.0244
MAE: 7.1301
R-Square: 0.1652
MSE: 2.3080
RMSE: 1.5192
MAE: 0.8156
R-Square: 0.3178
MSE: 9.4244
RMSE: 3.0699
MAE: 2.2007
R-Square: 0.9034
MSE: 0.3015
RMSE: 0.5491
MAE: 0.2484
R-S

MSE: 0.0212
RMSE: 0.1458
MAE: 0.0739
R-Square: 0.9740
MSE: 4.6876
RMSE: 2.1651
MAE: 0.9821
R-Square: 0.6498
MSE: 0.0488
RMSE: 0.2209
MAE: 0.1220
R-Square: 0.9402
2018_z_w3_ikey01_hourly_disk.csv
MSE: 34.8202
RMSE: 5.9009
MAE: 4.9556
R-Square: 0.0128
MSE: 34.7256
RMSE: 5.8928
MAE: 4.9515
R-Square: 0.0128
MSE: 6.7004
RMSE: 2.5885
MAE: 2.2058
R-Square: 0.8100
MSE: 6.6824
RMSE: 2.5850
MAE: 2.2034
R-Square: 0.8100
MSE: 16.7627
RMSE: 4.0942
MAE: 3.5773
R-Square: 0.5248
MSE: 16.7174
RMSE: 4.0887
MAE: 3.5735
R-Square: 0.5248
2018_z_w3_ikey01_hourly_mem.csv
MSE: 4.2308
RMSE: 2.0569
MAE: 0.2732
R-Square: -17.6435
MSE: 4.1445
RMSE: 2.0358
MAE: 0.2139
R-Square: -40.7707
MSE: 0.2042
RMSE: 0.4519
MAE: 0.0362
R-Square: 0.1002
MSE: 0.1956
RMSE: 0.4423
MAE: 0.0193
R-Square: -0.9712
MSE: 0.2460
RMSE: 0.4960
MAE: 0.0568
R-Square: -0.0842
MSE: 0.2355
RMSE: 0.4853
MAE: 0.0285
R-Square: -1.3732
